**Disclaimer:** some of the material is borrowed with thanks from http://www.cis.lmu.de/~hs/teach/14s/ir/

# Boolean model

* The Boolean model is arguably the simplest model to
  base an information retrieval system on.
* Queries are Boolean expressions, e.g., *Caesar* and *Brutus*
* The seach engine returns all documents that satisfy the Boolean expression, but there are exceptions:
  * anchor text
  * page contains variant of query words: morphology, spelling correction, synonyms
* No notion of ranking

## Term-document matrix

* Rows: terms
* Columns: document
* $M_{ij}=1$ if term *i* present in document *j*


In [114]:
from sklearn.feature_extraction.text import CountVectorizer
documents=["This is a silly example","A better example","Nothing to see here","This is a great and long example"]
cv=CountVectorizer(lowercase=False,binary=True)
print("Term-document matrix:\n")
td_matrix=cv.fit_transform(documents).todense().T   #.T transposes the matrix, sklearn maintains document-term
print(td_matrix)
print("\nIDX -> terms mapping:\n")
print(cv.get_feature_names())
print("\nterm -> IDX mapping:\n")
print(cv.vocabulary_) # note the _ at the end


Term-document matrix:

[[0 0 1 0]
 [1 0 0 1]
 [0 0 0 1]
 [0 1 0 0]
 [1 1 0 1]
 [0 0 0 1]
 [0 0 1 0]
 [1 0 0 1]
 [0 0 0 1]
 [0 0 1 0]
 [1 0 0 0]
 [0 0 1 0]]

IDX -> terms mapping:

['Nothing', 'This', 'and', 'better', 'example', 'great', 'here', 'is', 'long', 'see', 'silly', 'to']

term -> IDX mapping:

{'long': 8, 'see': 9, 'This': 1, 'Nothing': 0, 'is': 7, 'silly': 10, 'example': 4, 'better': 3, 'great': 5, 'here': 6, 'and': 2, 'to': 11}


* Every row is an *incidence vector* of a term - which documents the term appears in
* Boolean retrieval in its simplest form:
  * Boolean operations on incidence vectors

In [115]:
t2i=cv.vocabulary_
print("example")
print(td_matrix[t2i["example"]])
print()
print("example and great")
print(td_matrix[t2i["example"]] & td_matrix[t2i["great"]])
print()
print("not example")
print(1-td_matrix[t2i["example"]]) #1-x does the negation in our case
print()

example
[[1 1 0 1]]

example and great
[[0 0 0 1]]

not example
[[0 0 1 0]]



Now we can piece it all together:

* Accept queries like "( not example or great ) and Nothing"
* Rewrite them into a Python expression
* eval() that expression

In [116]:
# Operators and, or, not become &, |, 1 -
# Parentheses are left untouched
# Everything else interpreted as a term and fed through td_matrix[t2i["..."]]

d={"and":"&","or":"|","not":"1 -","(":"(",")":")"} # operator replacements
def rew_token(t):
    return d.get(t,'td_matrix[t2i["{}"]]'.format(t))

def rew_query(query): #rewrite every token in the query
    return " ".join(rew_token(t) for t in query.split())

def test_query(query):
    print("Query:\""+query+"\"")
    print("Rewritten:",rew_query(query))
    print("Matching:",eval(rew_query(query)))
    print()

test_query("example and not Nothing")
test_query("not example or great")
test_query("( not example or great ) and Nothing")


Query:"example and not Nothing"
Rewritten: td_matrix[t2i["example"]] & 1 - td_matrix[t2i["Nothing"]]
Matching: [[1 1 0 1]]

Query:"not example or great"
Rewritten: 1 - td_matrix[t2i["example"]] | td_matrix[t2i["great"]]
Matching: [[0 0 1 1]]

Query:"( not example or great ) and Nothing"
Rewritten: ( 1 - td_matrix[t2i["example"]] | td_matrix[t2i["great"]] ) & td_matrix[t2i["Nothing"]]
Matching: [[0 0 1 0]]



In [117]:
# Should match all documents, but crashes instead
# This one you will fix as an exercise
import traceback
try:
    test_query("not awordwhichdoesnotexist") #should match all documents
except:
    traceback.print_exc() #I only need to do this because of IPython

Query:"not awordwhichdoesnotexist"
Rewritten: 1 - td_matrix[t2i["awordwhichdoesnotexist"]]


Traceback (most recent call last):
  File "<ipython-input-117-69ec74878d51>", line 5, in <module>
    test_query("not awordwhichdoesnotexist") #should match all documents
  File "<ipython-input-116-41857662fdc6>", line 15, in test_query
    print("Matching:",eval(rew_query(query)))
  File "<string>", line 1, in <module>
KeyError: 'awordwhichdoesnotexist'


* We now have a rudimentary boolean IR system
* It is not that great but ready in < 10 lines of code in Python

# Size constraints

* Term-document matrix is vocabulary size times document set size
* 1M documents, each 1000 words
* 1,000,000 x 1,000 = 1,000,000,000 words of running text
* 6 bytes per word -> ~6GB in size
* Assume 500,000 unique terms
* Term-document matrix has 1,000,000 x 500,000 / 8 / 1024 / 1024 / 1024 -> ~60GB
* 60GB of space to index a collection of 6GB of text!
* 480GB if we were to use 1B integers to remember the 0/1 values
* ...but most of this are zeros...

## Sparse representation

* Only remember the non-zero entries
* For every term, remember a (usually sorted) list of documents in which it appears
* This is the famous **inverted index**
* Scipy sparse formats: https://docs.scipy.org/doc/scipy-0.18.1/reference/sparse.html
  * *CSC* for every column remember the list of rows
  * *CSR* for every row remember the list of columns
  


In [118]:
documents=["This is a silly example","A better example","Nothing to see here","This is a great and long example"]
cv=CountVectorizer(lowercase=False,binary=True)
# Exact same code as above, but I removed the .todense()
td_matrix=cv.fit_transform(documents)
print("scikit's document-term matrix:")
print(td_matrix)
print()
print("Transposed: (note incorrect sort)")
print(td_matrix.T)
print()
print("Transposed, and in the correct sparse format:")
print(td_matrix.T.tocsr())
td_matrix=td_matrix.T.tocsr()

scikit's document-term matrix:
  (0, 1)	1
  (0, 7)	1
  (0, 10)	1
  (0, 4)	1
  (1, 4)	1
  (1, 3)	1
  (2, 0)	1
  (2, 11)	1
  (2, 9)	1
  (2, 6)	1
  (3, 1)	1
  (3, 7)	1
  (3, 4)	1
  (3, 5)	1
  (3, 2)	1
  (3, 8)	1

Transposed: (note incorrect sort)
  (1, 0)	1
  (7, 0)	1
  (10, 0)	1
  (4, 0)	1
  (4, 1)	1
  (3, 1)	1
  (0, 2)	1
  (11, 2)	1
  (9, 2)	1
  (6, 2)	1
  (1, 3)	1
  (7, 3)	1
  (4, 3)	1
  (5, 3)	1
  (2, 3)	1
  (8, 3)	1

Transposed, and in the correct sparse format:
  (0, 2)	1
  (1, 0)	1
  (1, 3)	1
  (2, 3)	1
  (3, 1)	1
  (4, 0)	1
  (4, 1)	1
  (4, 3)	1
  (5, 3)	1
  (6, 2)	1
  (7, 0)	1
  (7, 3)	1
  (8, 3)	1
  (9, 2)	1
  (10, 0)	1
  (11, 2)	1


In [119]:
# The sparse representations do not allow many of the necessary operations, so we need
# to make the rows dense, once we retrieve them, not a huge deal for our toy examples
def rew_token(t):
    return d.get(t,'td_matrix[t2i["{}"]].todense()'.format(t))

test_query("example and not Nothing")
test_query("( not example or great ) and Nothing")

Query:"example and not Nothing"
Rewritten: td_matrix[t2i["example"]].todense() & 1 - td_matrix[t2i["Nothing"]].todense()
Matching: [[1 1 0 1]]

Query:"( not example or great ) and Nothing"
Rewritten: ( 1 - td_matrix[t2i["example"]].todense() | td_matrix[t2i["great"]].todense() ) & td_matrix[t2i["Nothing"]].todense()
Matching: [[0 0 1 0]]



## Boolean retrieval (cont.)

* The code above needs `.todense()` to perform the *and / or / not* arithmetics -> inefficient, but simple
* Other option - make sure the lists of documents are sorted
* AND - intersection of two sorted lists
  * Walk the lists (I'll show on the lecture how - it's rather obvious anyway)
  * Linear complexity in terms of number of documents
  * A good exercise - write a function which takes two lists like those above, and computes their intersection as a new list

In [120]:
print("Documents for 'example'")
print(td_matrix[t2i["example"]].nonzero()[1])
print()
print("Documents for 'great'")
print(td_matrix[t2i["great"]].nonzero()[1])

Documents for 'example'
[0 1 3]

Documents for 'great'
[3]


# Phrase and proximity queries

* *"Stanford university"* - not the same thing as *Stanford AND university*
* The basic inverted index of no help

## biword index

* Index all word bigrams
* "Stanford university" becomes a term
* "to be or not to be" -> "to be" and "be or" and "or not" and "not to" + postprocessing
* Not a great solution:
  * Massive waste of space: blows up the index size quadratically
  * Needs postprocessing to weed out the false hits

## positional index

* For every term, and every document, remember a list of the positions, not just "1"
* A modification of the sorted list intersection algorithm to answer the query
* Also allows proximity queries "X within N words from Y"
* Quite heavy computationally

## combined index

* Most common biwords indexed directly
* Rest solved with positional indexing
* Compromise between the two

# Result ranking

* Obviously useful for large document collections
* Come up with a number describing the fit of a document to a query
* Return top-N documents
* Basic observations:
  * The more query terms hit, the more relevant the document is
  * The more times the query terms hit in the document, the more relevant the document is
  * Rare terms are more informative than common terms
* Firstly, we don't want to store 0/1, we want to store the count

In [121]:
documents=["This is a silly silly silly example","A better example","Nothing to see here nor here nor here","This is a great example and a long example too"]
cv=CountVectorizer(lowercase=False)
# Exact same code as above, but now I also removed binary=True
td_matrix=cv.fit_transform(documents)
t2i=cv.vocabulary_
td_matrix=td_matrix.T.tocsr()
print(td_matrix.todense())

[[0 0 1 0]
 [1 0 0 1]
 [0 0 0 1]
 [0 1 0 0]
 [1 1 0 2]
 [0 0 0 1]
 [0 0 3 0]
 [1 0 0 1]
 [0 0 0 1]
 [0 0 2 0]
 [0 0 1 0]
 [3 0 0 0]
 [0 0 1 0]
 [0 0 0 1]]
